#Peptide sequence predictor based on Rachel/Jeremy's awesome [Code-First Intro to Natural Language Processing](https://www.youtube.com/playlist?list=PL8l5P33wvCvIqO3UyvdkeRMoGg--VBvho) and respective [Notebook](https://github.com/fastai/course-nlp/)

<a href="https://colab.research.google.com/github/animesh/notebooks/blob/master/pepSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.text import *

In [13]:
#bs=12
bs=48
#bs=128

In [ ]:
torch.cuda.set_device(0)

In [2]:
data_path = Config.data_path()
print(data_path)

/home/ash022/.fastai/data


This will create a `viwiki` folder, containing a `viwiki` text file with the wikipedia contents. (For other languages, replace `vi` with the appropriate code from the [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias).)

In [3]:
lang = 'peptides'
#lang = 'vi'
# lang = 'zh'

In [4]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
print(name,lm_fns,path)

peptideswiki ['peptides_wt', 'peptides_wt_vocab'] /home/ash022/.fastai/data/peptideswiki


In [5]:
%%bash
#https://refgenie.databio.org/en/latest/
pip install --user refgenie
#export PATH=$PATH:/home/ash022/.local/bin
export REFGENIE='genome_config.yaml'
#refgenie init -c $REFGENIE

  Using cached https://files.pythonhosted.org/packages/00/37/ec89398d3163f8f63d892328730e04b3a10927e3780af25baf1ec74f880f/oyaml-0.9-py2.py3-none-any.whl


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
%%bash
/home/ash022/.local/bin/refgenie listr
#refgenie pull --genome hg38 --asset bowtie2_index
#refgenie build --genome mygenome --asset bwa_index --fasta mygenome.fa.gz
#refgenie seek --genome mm10 --asset bowtie2_index


refgenie 0.6.0
Traceback (most recent call last):
  File "/home/ash022/.local/bin/refgenie", line 10, in <module>
    sys.exit(main())
  File "/home/ash022/.local/lib/python3.7/site-packages/refgenie/refgenie.py", line 404, in main
    raise MissingGenomeConfigError(args.genome_config)
refgenie.exceptions.MissingGenomeConfigError: You must provide a config file either as an argument or via an environment variable: ['REFGENIE']


CalledProcessError: Command 'b'/home/ash022/.local/bin/refgenie listr\n#refgenie pull --genome hg38 --asset bowtie2_index\n#refgenie build --genome mygenome --asset bwa_index --fasta mygenome.fa.gz\n#refgenie seek --genome mm10 --asset bowtie2_index\n'' returned non-zero exit status 1.

### Download data

In [ ]:
import urllib.parse
import urllib.request

#url = 'https://www.uniprot.org/uploadlists/'
#params = {'from': 'ACC+ID','to': 'ENSEMBL_ID','format': 'tab','query': 'P40925 P40926 O43175 Q9UM73 P97793'}
url = 'https://www.uniprot.org/uniprot/'
#params = {'reviewed': 'yes','taxonomy':'Homo sapiens (Human) [9606]', 'format':'fasta', 'limit': '10000'}
params = {'reviewed': 'yes','format':'fasta', 'limit': '10000'}

data = urllib.parse.urlencode(params)
data = data.encode('utf-8')
req = urllib.request.Request(url, data)
with urllib.request.urlopen(req) as f:
   response = f.read()

In [ ]:
#print(response.decode('utf-8'))

In [ ]:
with open(path / name, 'wb') as fo:
    fo.write(response)

In [7]:
name=path.ls()
print(name)

[PosixPath('/home/ash022/.fastai/data/peptideswiki/docs'), PosixPath('/home/ash022/.fastai/data/peptideswiki/peptideswiki'), PosixPath('/home/ash022/.fastai/data/peptideswiki/models'), PosixPath('/home/ash022/.fastai/data/peptideswiki/fasta.txt')]


In [8]:
#!head -n4 {name}
!head -n2 /home/ash022/.fastai/data/peptideswiki/peptideswiki

>sp|Q0ATK2|ACCD_MARMM Acetyl-coenzyme A carboxylase carboxyl transferase subunit beta OS=Maricaulis maris (strain MCS10) OX=394221 GN=accD PE=3 SV=1
MTEKSNGMSWLSKITPPGMSKIFSKRDTPDNLWVKCPVSEEMVFHKDLEAGLFVTPAGHH


In [9]:
def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    #if dest.exists():
    #    print(f"{dest} already exists; not splitting")
    #    return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'^>')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%1000 == 0: print(i)
        if l.startswith('>'):
            title = l#title_re.findall(l)[0].replace('>','_')
            title=title.replace('>','')
            title=title.replace('|','_')
            title=title.split(' ')[0]
            if len(title)>1500: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else:
            l=list(l)#.split('')
            l=' '.join(l)
            f.write(str(l))
    f.close()
    return dest

This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [10]:
dest = split_wiki(path,lang)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000


In [11]:
dest.ls()[:5]

[PosixPath('/home/ash022/.fastai/data/peptideswiki/docs/sp_Q8NNJ8_COBT_CORGL.txt'),
 PosixPath('/home/ash022/.fastai/data/peptideswiki/docs/sp_Q5HJJ1_ARGB_STAAC.txt'),
 PosixPath('/home/ash022/.fastai/data/peptideswiki/docs/sp_Q8RT67_COAD_BARBK.txt'),
 PosixPath('/home/ash022/.fastai/data/peptideswiki/docs/sp_P49419_AL7A1_HUMAN.txt'),
 PosixPath('/home/ash022/.fastai/data/peptideswiki/docs/sp_Q5HX60_ATPG_CAMJR.txt')]

### Create pretrained model

In [14]:
data = (TextList.from_folder(dest)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

(48, 9000)

In [15]:
print(path,f'{lang}_databunch')
#!mv /home/ash022/.fastai/data/peptidewiki/docs/peptide_databunch /home/ash022/.fastai/data/peptidewiki/peptide_databunch

/home/ash022/.fastai/data/peptideswiki peptides_databunch


In [16]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)#.to_fp16()

In [17]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [20]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,2.489825,2.482903,0.241727,01:05
1,2.520136,2.518637,0.229921,01:05
2,2.528511,2.548135,0.221293,01:05
3,2.546758,2.543422,0.220551,01:05
4,2.513374,2.511421,0.232049,01:05
5,2.481613,2.478046,0.242860,01:05
6,2.418326,2.446834,0.251266,01:05
7,2.444733,2.417113,0.261829,01:05
8,2.339745,2.401023,0.266787,01:05
9,2.264809,2.398211,0.268027,01:05


Save the pretrained model and vocab:

In [ ]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

## spectra analysis

### data

- [What are you inferring?](http://www.matrixscience.com/blog/what-are-you-inferring.html)
- [Human Phosphopeptide Label Free Library](https://chemdata.nist.gov/dokuwiki/doku.php?id=peptidew:lib:phoshopept_labelfree_20190214)
- [A protein standard that emulates homology for the characterization of protein inference algorithms](https://www.ebi.ac.uk/pride/archive/projects/PXD008425)

In [ ]:
pos_df = pd.read_table('mgf.txt',header=None)
pos_df.columns=['comment']
pos_df['label']=1
#train_df.loc[pd.isna(train_df.comment),'comment']='NA'
pos_df.head()

In [ ]:
neg_df = pd.read_table('mgf_test.txt',header=None,sep="\n")
neg_df.columns=['comment']
neg_df['label']=0
#test_df.loc[pd.isna(test_df.comment),'comment']='NA'
neg_df.head()

In [ ]:
df = pd.concat([pos_df,neg_df], sort=False)
df.head(), df.tail()

In [ ]:
data_lm = (TextList.from_df(df, path, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

In [ ]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [ ]:
lr = 1e-3
lr *= bs/48

In [ ]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

In [ ]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(8, lr, moms=(0.8,0.7))

In [ ]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [ ]:
data_clas = (TextList.from_df(pos_df, path, vocab=data_lm.vocab, cols='comment')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='label')
    .databunch(bs=bs, num_workers=1))

data_clas.save(f'{lang}_textlist_class')

In [ ]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)

In [ ]:
from sklearn.metrics import f1_score
import numpy as np
#@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))

In [ ]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [ ]:
lr=2e-2
lr *= bs/48

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7)).to_fp16()

In [ ]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

In [ ]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

In [ ]:
learn_c.save(f'{lang}clas')

Competition top 3 f1 scores: 0.90, 0.89, 0.89. Winner used an ensemble of 4 models: TextCNN, VDCNN, HARNN, and SARNN.

## Ensemble

In [ ]:
data_clas = load_data(path, f'{lang}_textlist_class', bs=bs, num_workers=1)
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c.load(f'{lang}clas', purge=False);

In [ ]:
preds,targs = learn_c.get_preds(ordered=True)
accuracy(preds,targs),f1(preds,targs)

In [ ]:
data_clas_bwd = load_data(path, f'{lang}_textlist_class_bwd', bs=bs, num_workers=1, backwards=True)
learn_c_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1]).to_fp16()
learn_c_bwd.load(f'{lang}clas_bwd', purge=False);

In [ ]:
preds_b,targs_b = learn_c_bwd.get_preds(ordered=True)
accuracy(preds_b,targs_b),f1(preds_b,targs_b)

In [ ]:
preds_avg = (preds+preds_b)/2

In [ ]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

## Future work
* Try [SentencePiece](https://github.com/google/sentencepiece)
* Check activation with [Interpretability Beyond Feature Attribution: Quantitative Testing with Concept Activation Vectors](https://github.com/tensorflow/tcav)
* [GPT2](https://gist.github.com/mohdsanadzakirizvi/f1419fba3907af2baf6e0f6ab2a53d5b)
* [Unsupervised word embeddings capture latent knowledge from materials science literature](https://www.nature.com/articles/s41586-019-1335-8)
